# SSL Connection Examples

## Connecting to a Redis instance via SSL

In [ ]:
import redis

r = redis.Redis(
    host='localhost', 
    port=6666, 
    ssl=True, 
    ssl_cert_reqs="none",
)
r.ping()

True

## Connecting to a Redis instance via a URL string

In [ ]:
import redis

r = redis.from_url("rediss://localhost:6666?ssl_cert_reqs=none&decode_responses=True&health_check_interval=2")
r.ping()

True

## Connecting to a Redis instance using a ConnectionPool

In [ ]:
import redis

redis_pool = redis.ConnectionPool(
    host="localhost", 
    port=6666, 
    connection_class=redis.SSLConnection, 
    ssl_cert_reqs="none",
)

r = redis.StrictRedis(connection_pool=redis_pool) 
r.ping()

True

## Connecting to a Redis instance via SSL, while specifying a minimum TLS version

In [ ]:
import redis
import ssl

r = redis.Redis(
    host="localhost",
    port=6666,
    ssl=True,
    ssl_min_version=ssl.TLSVersion.TLSv1_3,
    ssl_cert_reqs="none",
)
r.ping()

True

## Connecting to a Redis instance via SSL, while specifying a self-signed SSL CA certificate

In [ ]:
import os
import redis

pki_dir = os.path.join("..", "..", "dockers", "stunnel", "keys")

r = redis.Redis(
    host="localhost",
    port=6666,
    ssl=True,
    ssl_certfile=os.path.join(pki_dir, "client-cert.pem"),
    ssl_keyfile=os.path.join(pki_dir, "client-key.pem"),
    ssl_cert_reqs="required",
    ssl_ca_certs=os.path.join(pki_dir, "ca-cert.pem"),
)
r.ping()

True

## Connecting to a Redis instance via SSL, and validate the OCSP status of the certificate

The redis package is designed to be small, meaning extra libraries must be installed, in order to support OCSP stapling. As a result, first install redis via:

`pip install redis[ocsp]`

This will install cryptography, requests, and PyOpenSSL, none of which are generally required to use Redis.

In the next example, we will connect to a Redis instance via SSL, and validate the OCSP status of the certificate. However, the certificate we are using does not have an AIA extension, which means that the OCSP validation cannot be performed.

In [ ]:
import os
import redis

pki_dir = os.path.join("..", "..", "dockers", "stunnel", "keys")

r = redis.Redis(
    host="localhost",
    port=6666,
    ssl=True,
    ssl_certfile=os.path.join(pki_dir, "client-cert.pem"),
    ssl_keyfile=os.path.join(pki_dir, "client-key.pem"),
    ssl_cert_reqs="required",
    ssl_ca_certs=os.path.join(pki_dir, "ca-cert.pem"),
    ssl_validate_ocsp=True,
)

try:
    r.ping()
except redis.ConnectionError as e:
    assert e.args[0] == "No AIA information present in ssl certificate"
    print("OCSP validation failed as expected.")

OCSP validation failed as expected.


## Connect to a Redis instance via SSL, and validate OCSP-stapled certificates

It is also possible to validate an OCSP stapled response. Again, for this example the server does not send an OCSP stapled response, so the validation will fail.

In [ ]:
import os
import redis

pki_dir = os.path.join("..", "..", "dockers", "stunnel", "keys")
ca_cert = os.path.join(pki_dir, "ca-cert.pem")

# It is possible to specify an expected certificate, or leave it out.
expected_certificate = open(ca_cert, 'rb').read()

# If needed, a custom SSL context for OCSP can be specified via ssl_ocsp_context

r = redis.Redis(
    host="localhost",
    port=6666,
    ssl=True,
    ssl_certfile=os.path.join(pki_dir, "client-cert.pem"),
    ssl_keyfile=os.path.join(pki_dir, "client-key.pem"),
    ssl_cert_reqs="required",
    ssl_ca_certs=ca_cert,
    ssl_validate_ocsp_stapled=True,
    ssl_ocsp_expected_cert=expected_certificate,
)

try:
    r.ping()
except redis.ConnectionError as e:
    assert e.args[0] == "no ocsp response present"
    print("OCSP validation failed as expected.")

OCSP validation failed as expected.
